In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow_text
import tensorflow_text as text

# !pip install -q tf-models-official==2.7.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.6 MB/s eta 0:00:00


In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import datetime

# from official.nlp import optimization  # to create AdamW optimizer


tf.get_logger().setLevel('ERROR')

In [ ]:
glue, info = tfds.load('glue/sst2', with_info=True, batch_size=-1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/872 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/sst2/2.0.0.incomplete5PA6V1/glue-validation.tfrecord*...:   0%|      …

Generating test examples...:   0%|          | 0/1821 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/sst2/2.0.0.incomplete5PA6V1/glue-test.tfrecord*...:   0%|          | …

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/sst2/2.0.0. Subsequent calls will reuse this data.


In [ ]:
glue.keys()

dict_keys([Split('train'), Split('validation'), Split('test')])

In [ ]:
x_train = glue['train']['sentence']
y_train = glue['train']['label']

x_val = glue['validation']['sentence']
y_val = glue['validation']['label']


In [ ]:
glue['train']

{'idx': <tf.Tensor: shape=(67349,), dtype=int32, numpy=array([16399,  1680, 47917, ..., 39601, 32842, 50006], dtype=int32)>,
 'label': <tf.Tensor: shape=(67349,), dtype=int64, numpy=array([0, 0, 1, ..., 1, 0, 0])>,
 'sentence': <tf.Tensor: shape=(67349,), dtype=string, numpy=
 array([b'for the uninitiated plays better on video with the sound ',
        b'like a giant commercial for universal studios , where much of the action takes place ',
        b'company once again dazzle and delight us ', ...,
        b'more playful ',
        b'the very definition of what critics have come to term an `` ambitious failure ',
        b'barn-burningly bad movie '], dtype=object)>}

In [ ]:
glue['train']['sentence'].numpy()

array([b'for the uninitiated plays better on video with the sound ',
       b'like a giant commercial for universal studios , where much of the action takes place ',
       b'company once again dazzle and delight us ', ...,
       b'more playful ',
       b'the very definition of what critics have come to term an `` ambitious failure ',
       b'barn-burningly bad movie '], dtype=object)

In [ ]:
len(y_train) // 32

2104

In [ ]:
preprocessor_handle = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
# model_handle = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1"
model_handle = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1"

In [ ]:
preprocessor = hub.KerasLayer(preprocessor_handle)

In [ ]:
inp = preprocessor(["hello"])

In [ ]:
inp

{'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[ 101, 7592,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(1, 128

In [ ]:
transformer_encoder = hub.KerasLayer(model_handle, trainable=False)

In [ ]:
class TextClassificationModel(tf.keras.Model):
  def __init__(self, preprocessor_handle, model_handle):
    super().__init__()
    self._preprocessor = hub.KerasLayer(preprocessor_handle)
    self._transformer_encoder = hub.KerasLayer(model_handle, trainable=True)
    self._classifier = tf.keras.layers.Dense(units=1, activation=None)

  def call(self, inputs):
    tokenized_output = self._preprocessor(inputs)
    embedding = self._transformer_encoder(tokenized_output)['pooled_output']
    logits = self._classifier(embedding)
    return logits

In [ ]:
model = TextClassificationModel(preprocessor_handle, model_handle)

In [ ]:
init_lr = 3e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=10000,
#                                           num_warmup_steps=1000,
#                                           optimizer_type='adamw')
optimizer = tf.keras.optimizers.Adam(3e-5)

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=[tf.metrics.BinaryAccuracy()]
)

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(x=x_train, y=y_train, batch_size=32, epochs=5, 
          validation_data=(x_val, y_val), 
          callbacks=[tensorboard_callback])

Epoch 1/5
2105/2105 [==============================] - 1183s 558ms/step - loss: 0.4415 - binary_accuracy: 0.7781 - val_loss: 0.4331 - val_binary_accuracy: 0.8039
Epoch 2/5
2105/2105 [==============================] - 1159s 550ms/step - loss: 0.2997 - binary_accuracy: 0.8680 - val_loss: 0.4341 - val_binary_accuracy: 0.8096
Epoch 3/5
2105/2105 [==============================] - 1168s 555ms/step - loss: 0.2466 - binary_accuracy: 0.8959 - val_loss: 0.4544 - val_binary_accuracy: 0.8154
Epoch 4/5
2105/2105 [==============================] - 1153s 548ms/step - loss: 0.2132 - binary_accuracy: 0.9129 - val_loss: 0.4590 - val_binary_accuracy: 0.8200
Epoch 5/5
2105/2105 [==============================] - 1136s 540ms/step - loss: 0.1884 - binary_accuracy: 0.9232 - val_loss: 0.5025 - val_binary_accuracy: 0.8257


In [ ]:
model.save_weights('./checkpoints2/my_checkpoint2')

In [ ]:
# Create a new model instance
model = model = TextClassificationModel(preprocessor_handle, model_handle)

# Restore the weights
model.load_weights('./checkpoints2/my_checkpoint2')

In [ ]:
x = model.call(["I will never watch this again", "I liked it", "This not giving value to the time", "I should not have watch this", "I should watch this much earlier"])

In [ ]:
results = []
print(x.numpy())
for i in x.numpy():
  results.append("Good") if i>=0 else results.append("Bad")
print(results)

[[ 0.57426506]
 [ 4.2697887 ]
 [-1.2688649 ]
 [-0.9724747 ]
 [-0.20022897]]
['Good', 'Good', 'Bad', 'Bad', 'Bad']


In [ ]:
x